In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from IngestionService import APIRequest, APIData
from Process import ProcessData
import os
import sqlalchemy
from sqlalchemy import create_engine

#bucket and bucket objects
bucket = os.getenv('bucket')
bucketobj = os.getenv('bucketobj')

# Class to get data from S3
dataClient = APIData(bucket=bucket, ACCESS_KEY=os.getenv('ACCESS_KEY'), SECRET_KEY=os.getenv('SECRET_KEY'))

# Class to clean data from S3 and put into RDS
clean = ProcessData(bucket=bucket, ACCESS_KEY=os.getenv('ACCESS_KEY'), SECRET_KEY=os.getenv('SECRET_KEY'))

# Data comes down from s3 in bytes fromat
s3response = clean.download_data(bucket, bucketobj)

# Data into dictionary format from bytes format
decode = json.loads(s3response.decode('utf-8'))

# data in list format
dirty = decode['fuel_stations']

# Returns dataframe of data
ready = clean.dict_df(dirty)

# DEFINE THE DATABASE CREDENTIALS
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
database = os.getenv('database')
port = os.getenv('port')
tablename = os.getenv('tablename')

df = ready[['latitude', 'longitude', 'city','intersection_directions', 'plus4', 'state', 'street_address', 'zip','country']]

up = clean.upload_data(user, password, host, database, port, tablename, df)

def lambda_handler(event, context):
    return {
        'statusCode': 200,
    }